In [202]:
import glob
import pandas as pd

In [203]:
# Criterios de coincidencia de patrones para archivos de Excel en el directorio
filenames = 'Cuenca*.xlsx'

In [204]:
# Crea un diccionario vacío para almacenar los datos de cada archivo
dat = {}

# Recorra cada archivo de Excel que coincida con el patrón y lea sus datos en un marco de datos de pandas
for file in glob.glob(filenames):
    dat[file] = pd.read_excel(file)

In [205]:
# Access the data from each file using its file name as the key in the data_dict
data_new = []

for file, data in dat.items():
    data_new.append(data)

In [206]:
# Definir una función para aplicar a los datos y en la columna 'ESTACION' todos los valores tengan 3 digitos
for i, df in enumerate(data_new):
    def formatear_valor(valor):
        if valor < 10:
            return '00' + str(valor)
        elif valor < 100:
            return '0' + str(valor)
        else:
            return str(valor)

    # Aplicar la función a la tabla
    data_new[i]['ESTACION'] = data_new[i]['ESTACION'].apply(formatear_valor)

In [207]:
#Unir el numero de 'CUENCA' con 'ESTACION' para tener el numero de cada estacion
for i, df in enumerate(data_new):
    data_new[i]['ESTACION_NEW'] = data_new[i]['CUENCA'].astype(str) + data_new[i]['ESTACION'].astype(str)

In [208]:
#Eliminar las columnas que no son necesarias
for i, df in enumerate(data_new):
    data_new[i].pop('CUENCA')
    data_new[i].pop('ESTACION')

In [209]:
len(data_new)

22

In [210]:
# Agrupar las filas según los valores de la columna "ESTACION_NEW"
my_list = list(range(0, 22))
porestacion=[]
for i in my_list:
    grupos = data_new[i].groupby('ESTACION_NEW')

    # Crear una tabla separada para cada grupo segun 'ESTACION_NEW'
    for ESTACION, grupo in grupos:
        # Obtener el DataFrame correspondiente a cada grupo
        tabla = grupos.get_group(ESTACION)
        porestacion.append(tabla) #Adjuntar los valores

In [211]:
NUMERO = porestacion.copy()
for i, df in enumerate(NUMERO):
    NUMERO[i] = NUMERO[i].iloc[0,2]

In [212]:
ESTACION = []
for i, df in enumerate(ESTACION):
    ESTACION[i] = 'Estacion' + NUMERO[i]

In [213]:
#largo del archivo
len(porestacion)

154

In [214]:
#Guardar cada tabla de cada estación en un CSV diferente
my_list = list(range(0, 154))
for a, i in zip(ESTACION,my_list):
    porestacion[i].to_csv(f'{a}.csv', index=True)

In [215]:
for i, df in enumerate(porestacion):
    porestacion[i] = porestacion[i].set_index('FECHA')

In [216]:
# Encontrar los valores de inicio y final de las fechas
my_list = list(range(0, 154))
porestacion_new=[]
for i in my_list:
    fecha_inicial = porestacion[i].index.min()
    fecha_final = porestacion[i].index.max()
    name = data2[i]
    porestacion_new.append((fecha_inicial,fecha_final, name))

In [217]:
#Guardar el rango de fechas en un archivo .txt
# Abrir el archivo en modo escritura
with open("rango_fechas.txt", "w") as archivo:
    # Escribir la lista en el archivo
    for valor in porestacion_new:
        archivo.write(str(valor) + "\n")

In [218]:
#Eliminar las columnas que no se necesitan
for i, df in enumerate(data_new):
    porestacion[i].pop('ESTACION_NEW')

In [219]:
#Obtener el promedio diario
porestacion_mean = []
for i, df in enumerate(porestacion):
    value = porestacion[i].groupby(porestacion[i].index.strftime("%m%d")).mean() 
    #groupby() se usa agrupando por mes y por día a la vez
    porestacion_mean.append(value)

In [220]:
#Seleccionar solo la columna que se quiere
for i, df in enumerate(porestacion_mean):
    porestacion_mean[i] = porestacion_mean[i].iloc[:,0]

In [221]:
#Volver un dataframe
porestacion_mean_new = pd.DataFrame(porestacion_mean)

In [222]:
#Transponer los valores
porestacion_mean_new = porestacion_mean_new.T

In [223]:
#Cambiar el nombre de las columnas
porestacion_mean_new.columns = NUMERO

In [224]:
porestacion_mean_new

,100633,100641,100643,100649,100651,100653,100655,200002,200007,200009,...,94019,96003,98075,98087,98091,98095,98097,98105,98107,98109
FECHA,,,,,,,,,,,,,,,,,,,,,
0101,1.850000,1.053846,2.984615,2.555556,2.400000,0.688889,0.250000,3.650000,0.57500,2.52,...,0.325,2.500000,0.195000,1.377778,0.604762,0.940,2.821429,0.03750,0.266667,1.12
0102,3.267647,1.353846,2.023077,6.800000,0.177778,3.322222,2.700000,17.562857,3.32500,42.12,...,3.275,0.844444,1.440000,0.157895,0.785714,1.125,7.978571,0.00000,9.533333,9.62
0103,5.308824,1.269231,4.376923,2.577778,4.800000,1.088889,4.850000,15.672857,1.06500,30.70,...,0.900,7.311111,1.770000,3.326316,1.576190,2.745,7.150000,1.71375,4.233333,8.98
0104,2.405882,5.515385,1.415385,4.155556,2.777778,0.533333,3.550000,13.911429,0.05000,11.92,...,0.200,2.988889,0.863158,1.778947,0.990476,2.725,3.592857,1.71250,4.233333,2.64
0105,2.152941,2.007692,6.061538,2.200000,22.622222,7.400000,1.225000,7.085714,0.32500,11.96,...,0.150,3.433333,2.642105,2.210526,1.738095,4.495,4.485714,4.40000,0.666667,0.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1227,5.734706,7.535714,4.771429,18.500000,9.620000,3.388889,5.422222,4.368333,4.84000,3.98,...,6.980,2.788889,2.610526,5.850000,2.040000,0.890,5.660000,0.21250,1.400000,1.96
1228,5.514706,1.157143,11.271429,4.033333,9.080000,3.255556,2.955556,2.125000,0.63125,1.10,...,3.900,0.622222,1.410526,1.044444,1.815000,0.695,5.366667,0.71250,7.566667,10.02
1229,3.857059,5.921429,1.364286,2.155556,6.320000,7.555556,3.822222,0.515000,2.02500,0.26,...,0.560,3.855556,1.431579,3.816667,0.740000,1.565,4.186667,3.36250,4.500000,8.26


In [225]:
#Cargar la lista de estaciones que contienen el rango de fechas
estaciones = pd.read_excel(r'/Users/arias/OneDrive/Documentos/UCR/TFG/Datos/Observada/Lluvia/Lista de estaciones.xlsx')

In [226]:
estaciones

,NUMERO,NOMBRE,LATITUD NORTE,LONGITUD OESTE,ALTITUD (msnm),INICIO,FINAL,ESTACION
0,100633,FINCA EL PATIO. PUERTO JIMENEZ,"08º 36' 04.6""","83º25'52.2""",24,2004-02-12,2022-12-31,100633
1,100641,COOPEAGROPAL. LAUREL .,"08º 28' 17.6""","82º51'38.4""",38,2007-03-09,2022-12-31,100641
2,100643,RIO CLARO,"08º 40' 26.5""","83º03'43.2""",51,2009-12-11,2022-12-31,100643
3,100649,FUNDACION NEOTROPICA. RINCON DE OSA,"08º 42' 2.9""","83º30'49.4""",80,2013-01-16,2022-11-10,100649
4,100651,COTO 49. CIUDAD NEILY,"08º 37' 52.3""","82º58'00.7""",30,2013-05-16,2022-12-31,100651
...,...,...,...,...,...,...,...,...
149,98095,ALTAMIRA,"09º 01' 45.7""","83º00'28.7""",1370,2002-01-01,2022-12-31,98095
150,98097,MONTECARLO. PZ,"09º 21' 28""","83º36'03""",1005,2008-06-17,2022-12-31,98097
151,98105,PLANTA DE VOLCAN. BUENOS AIRES,"09º 12' 15""","83º27'2.49""",416,2013-11-26,2022-12-07,98105
152,98107,CAFETALERA EL INDIO. SAN VITO. PUNTARENAS,"08º 49' 57.9""","82º57'10.5""",1027,2016-12-15,2022-11-10,98107
